# Extract data from subreddits 

### Imports

In [57]:
import praw 
import pandas as pd
import matplotlib as m
from dotenv import load_dotenv
from pathlib import Path
import os

### Loading creds from .env

In [58]:
load_dotenv()

client_id = os.getenv("REDDIT_CLIENT_ID")
client_secret = os.getenv("REDDIT_CLIENT_SECRET")
user_agent = os.getenv("REDDIT_USER_AGENT")

### Connecting to Reddit API

In [59]:
reddit = praw.Reddit(
    client_id = client_id,
    client_secret = client_secret,
    user_agent = user_agent
)

## Testing connection
subreddit = reddit.subreddit("dataengineering")
# print(f"Subreddit Title: {subreddit.title}")
# print(f"Subreddit Description: {subreddit.description}")
# print(f"Subreddit Subscribers: {subreddit.subscribers}")

### Ingest Data from top 10 posts for this week

In [60]:
posts = subreddit.top(time_filter = "week", limit=10)

data = []

for post in posts:
    data.append({
        "title": post.title,
        "score": post.score,
        "url": post.url,
        "num_comments": post.num_comments,
        "created_utc": post.created_utc
    })

# Print the data
# df = pd.DataFrame(data)
# df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')
# print(df[['title','score','num_comments','url']])

### Convert to .csv format and save to Bronze Layer

In [61]:
BASE_DIR = Path().resolve().parent  

output_path = BASE_DIR / 'data' / 'bronze' / 'raw_reddit_posts.csv'

df.to_csv(output_path, index=False)